In [ ]:
pip install tensorflow opencv-python

In [ ]:
# Directory containing your training images and labels
#train_data_directory = 'https://drive.google.com/drive/folders/16cEKULnazJsSyPzIZCSo2I6xjcCr80FL?usp=drive_link'

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Function to extract box count from filename
def extract_box_count(filename):
    # Assuming the count is before or after "boxes" in the filename
    parts = filename.lower().split('_')
    for part in parts:
        if 'boxes' in part:
            try:
                return int(part)
            except ValueError:
                pass
    return 0  # Default to 0 if no count found

In [ ]:
#CNN(Convolutional Neural Network)
#Pooling Layer - MaxPooling 2D

In [ ]:
# Function to create the counting model
def create_counting_model(input_shape=(224, 224, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))  # Linear activation for regression
    return model

In [ ]:
# Compile the model
model = create_counting_model()
model.compile(optimizer='adam', loss='mean_squared_error')  # Use mean squared error for counting

In [ ]:
# Directory containing your training images
train_data_directory = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures'
#https://drive.google.com/drive/folders/16cEKULnazJsSyPzIZCSo2I6xjcCr80FL?usp=drive_link

In [ ]:
# Define the number of epochs
epochs = 5

# Training loop
for epoch in range(epochs):
    for filename in os.listdir(train_data_directory):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(train_data_directory, filename)

            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))
            image = image / 255.0  # Normalize pixel values

            # Reshape the data to match the model input shape
            image = np.expand_dims(image, axis=0)
            count = np.array([[extract_box_count(filename)]])  # Extract count from filename

            # Train the model on the current image and count
            model.train_on_batch(image, count)

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load your pre-trained model (replace 'your_model_path' with the actual path)
model = tf.keras.models.load_model('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')


In [ ]:
# Function to count inventory boxes in an image
def count_boxes(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match the model input size
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Predict the count using the trained model
    prediction = model.predict(image)
    count = int(np.round(prediction[0][0]))

    return count


In [ ]:
# Example usage
image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/20231025_145833.jpg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 44ms/step
The predicted box count is: 0


In [ ]:
# Example usage
image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/20231025_145833.jpg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 49ms/step
The predicted box count is: 0


In [ ]:
# Function to count inventory boxes in an image
def count_boxes(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match the model input size
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Predict the count using the trained model
    prediction = model.predict(image)

    # Print raw prediction values
    raw_prediction = prediction[0]
    print(f'Raw prediction values: {raw_prediction}')

    count = int(np.round(raw_prediction))

    return count

# Example usage
image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/IMG-20231103-WA0049.jpeg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 75ms/step
Raw prediction values: [0.00244552]
The predicted box count is: 0


In [ ]:
# Import necessary libraries
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection import model_lib_v2

# Define paths and configurations
pipeline_config_path = '/content/drive/MyDrive/M_P Dataset csv/pipeline.config'
model_dir = ''

# Load pipeline config
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']

# Load custom configurations if needed
# ...

# Optionally, change some configurations
# model_config.ssd.num_classes = your_number_of_classes
# ...

# Save updated configurations
config_util.save_pipeline_config(configs, pipeline_config_path)

# Train the model
print("Training started...")
model_lib_v2.train_loop(
    pipeline_config_path=pipeline_config_path,
    model_dir=model_dir,
    train_steps=your_train_steps,  # Specify the number of training steps
    use_tpu=False,  # Set to True if using TPU
    checkpoint_every_n=your_checkpoint_frequency,  # Specify how often to save checkpoints
)

print("Training finished.")


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the exported Faster R-CNN model
model = tf.saved_model.load('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')     #your_exported_model_path/saved_model

# Function to count inventory boxes in an image using Faster R-CNN
def count_boxes_faster_rcnn(image_path):
    image = cv2.imread(image_path)
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)

    # Convert image to tensor
    input_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)

    # Run inference
    detections = model(input_tensor)

    # Count the number of detected boxes
    box_count = detections['num_detections'][0].numpy()

    return int(box_count)

In [ ]:
# Example usage
image_path = 'path/to/your/image.jpg'
box_count = count_boxes_faster_rcnn(image_path)
print(f'The predicted box count is: {box_count}')